In [ ]:
# 전이학습

## Pretrained Network(기학습 네트워크)

In [1]:
# 전이학습 모델로 VGG16 사용할 예정
#   VGG16 : 전이학습 모델 중 가장 심플한 모델
# 누군가 이미 만들어놓은 VGG16 이라는 이름의 CNN 모델을 이용하자.

from tensorflow.keras.applications import VGG16

model_base = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(150, 150, 3))
  # weights='imagenet'
  #   imagenet의 데이터들을 가지고,
  #   VGG16모델을 거쳐서 나온 값들을 weights에 저장해서 가져오라는 의미
  # include_top=False
  #   Top 부분은 우리가 직접 개, 고양이 데이터를 넣어야 하므로,
  #   Top 데이터는 가져올 필요 없다.

print(model_base.summary())
# 출력
  # Total params: 14,714,688
  # Trainable params: 14,714,688
  # Non-trainable params: 0
  # 학습되어있는 데이터 1400만개 보유하게 됨

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/content/drive/MyDrive/한국SW산업협회 교육-Java 과정/파이썬 실습/cat_dog_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1/255)
batch_size=20

def extract_feature(directory, sample_count):
    features = np.zeros(shape=(sample_count,4,4,512))
    labels = np.zeros(shape=(sample_count,))
    
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode='binary')

    i = 0

    for x_data_batch, t_data_batch in generator:
        feature_batch = model_base.predict(x_data_batch)
        features[i*batch_size:(i+1)*batch_size] = feature_batch
        labels[i*batch_size:(i+1)*batch_size] = t_data_batch
        
        i += 1
        if i * batch_size >= sample_count:
            break;
    
    return features, labels

train_features, train_labels = extract_feature(train_dir,2000)
validation_features, validation_labels = extract_feature(validation_dir,1000)
test_features, test_labels = extract_feature(test_dir,1000)

Found 2000 images belonging to 2 classes.
1/1 [==============================] - 0s 21ms/step
Found 1000 images belonging to 2 classes.
1/1 [==============================] - 0s 29ms/step
Found 1000 images belonging to 2 classes.
1/1 [==============================] - 0s 22ms/step


In [3]:
# 위에서 이미지의 특징을 뽑아내서 학습할 데이터 생성

train_features = np.reshape(train_features, (2000,4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000,4 * 4 * 512))
test_features = np.reshape(test_features, (1000,4 * 4 * 512))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(Dense(256,
                activation='relu',
                input_shape=(4 * 4 * 512,)))
model.add(Dropout(0.5))
model.add(Dense(1,
                activation='sigmoid'))

model.compile(optimizer=RMSprop(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_features,
                    train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))


Epoch 1/30
100/100 [==============================] - 2s 6ms/step - loss: 0.5794 - accuracy: 0.6910 - val_loss: 0.4202 - val_accuracy: 0.8410
Epoch 2/30
100/100 [==============================] - 0s 4ms/step - loss: 0.4229 - accuracy: 0.8125 - val_loss: 0.3494 - val_accuracy: 0.8700
Epoch 3/30
100/100 [==============================] - 0s 4ms/step - loss: 0.3573 - accuracy: 0.8440 - val_loss: 0.3205 - val_accuracy: 0.8740
Epoch 4/30
100/100 [==============================] - 0s 4ms/step - loss: 0.3170 - accuracy: 0.8670 - val_loss: 0.2962 - val_accuracy: 0.8900
Epoch 5/30
100/100 [==============================] - 0s 4ms/step - loss: 0.2921 - accuracy: 0.8795 - val_loss: 0.2879 - val_accuracy: 0.8860
Epoch 6/30
100/100 [==============================] - 0s 4ms/step - loss: 0.2559 - accuracy: 0.8995 - val_loss: 0.2699 - val_accuracy: 0.8920
Epoch 7/30
100/100 [==============================] - 0s 4ms/step - loss: 0.2541 - accuracy: 0.9040 - val_loss: 0.2627 - val_accuracy: 0.8960
Epoch 